# Offline Policy Evaluation w/ minibatch

Testing offline policy evalation with zone-agnostic behavior policy model

In [1]:
import numpy as np
import sys
import matplotlib.pyplot as plt
import pandas as pd
import pickle
sys.path.insert(0, "../")
import torch
from ppo import PPO
import glob
from tqdm.notebook import tqdm
import json
from scipy.stats import spearmanr
import importlib

%matplotlib widget

In [2]:
log_data = pd.read_csv("../data/rule_based_log_data/0_cleaned_log.csv")
with open("../data/rule_based_log_data/action_probs_all_data.pkl", "rb") as f:
    behavior_model = pickle.load(f)
with open("../data/invalid_policy_list.json") as f:
    invalid_policies = json.load(f)["invalid_policies"]

## 30 Day MiniBatch

In [3]:
num_ts_per_day = 4 * 24
num_days = 30
ts_end = num_ts_per_day * num_days
zones = log_data["zone"].unique()

## Loading all Policies

In [4]:
policy_list = sorted(list(glob.glob(f"../policy_library/**.pth")))

### 1. Inverse Probability Weighting

In [5]:
from ope.iw import InverseProbabilityWeighting
policy_scores = {}
for zone in zones:
    print(zone)
    ope_data = log_data[log_data["zone"] == zone].sort_values(by=["timestep"])
    ope_data = log_data[:ts_end]
    ipw = InverseProbabilityWeighting(ope_data, retain_grad_fn=False, univariate_action=True)
    for policy in tqdm(policy_list):
        agent = PPO(6, 1, 0.003, 0.0005, 1, 10, 0.2,
                    has_continuous_action_space=True, action_std_init=0.2, 
                    device=torch.device('cpu'), diverse_policies=list(),
                    diverse_weight=0, diverse_increase=True)
        agent.load(policy)
        agent.policy_evaluation = True
        agent.policy_old.set_action_std(0.1)
        if policy not in invalid_policies:
            score, _, _, _, _ = ipw.evaluate_policy(agent.select_action, behavior_model, score="mean")
        else:
            continue
        if policy not in policy_scores:
            policy_scores[policy] = {}
        if zone not in policy_scores[policy]:
            policy_scores[policy][zone] = score.item()

Perimeter_top_ZN_3


  0%|          | 0/800 [00:00<?, ?it/s]

/Users/aakashsasikumar/Documents/MSc/Research/Dissimilar Policy/building-MARL/studies/../ope/iw.py:61: RuntimeWarning: divide by zero encountered in log
  return np.log(value / (1- value))


Perimeter_bot_ZN_1


  0%|          | 0/800 [00:00<?, ?it/s]

...

  0%|          | 0/800 [00:00<?, ?it/s]

Perimeter_mid_ZN_1


  0%|          | 0/800 [00:00<?, ?it/s]

Perimeter_bot_ZN_4


  0%|          | 0/800 [00:00<?, ?it/s]

In [6]:
eval_data_loc = "../data/1month_eval.csv"
df = pd.read_csv(eval_data_loc, header=None, names=["datetime","policy","zone","energy"])
spearman_corr = {}
for zone in zones:
    eval_df = df[df["zone"] == zone]

    # invalid_policies = list(set(policy_list) - set(policy_scores.keys()))
    for i_policy in invalid_policies:
        eval_df = eval_df[eval_df["policy"]!=i_policy[3:]]
    eval_df = eval_df.sort_values(by=["energy"])
    score_list = []
    for i, row in eval_df.iterrows():
        score_list.append(policy_scores[f"../{row['policy']}"][zone])

    eval_df["ope_scores"] = score_list
    correlation = spearmanr(eval_df["energy"].values, eval_df["ope_scores"].values)
    spearman_corr[zone] = correlation

In [22]:
with open("data/ipw_raw_scores_11_07_2022.pkl", "wb+") as f:
    pickle.dump(policy_scores, f)

In [8]:
with open("ipw_spearman_corrs_11_07_2022.pkl", "wb+") as f:
    pickle.dump(spearman_corr, f)

### 2. Self Normalized Inverse Probability Weighting

In [5]:
from ope.sniw import SelfNormalizedInverseProbabilityWeighting
policy_scores_sniw = {}
for zone in zones:
    print(zone)
    ope_data = log_data[log_data["zone"] == zone].sort_values(by=["timestep"])
    ope_data = log_data[:ts_end]
    snipw = SelfNormalizedInverseProbabilityWeighting(ope_data)
    for policy in tqdm(policy_list):
        agent = PPO(6, 1, 0.003, 0.0005, 1, 10, 0.2,
                    has_continuous_action_space=True, action_std_init=0.2, 
                    device=torch.device('cpu'), diverse_policies=list(),
                    diverse_weight=0, diverse_increase=True)
        agent.load(policy)
        agent.policy_evaluation = True
        agent.policy_old.set_action_std(0.1)
        if policy not in invalid_policies:
            score = snipw.evaluate_policy(agent.select_action, behavior_model, score="mean")
        else:
            continue
        if policy not in policy_scores_sniw:
            policy_scores_sniw[policy] = {}
        if zone not in policy_scores_sniw[policy]:
            policy_scores_sniw[policy][zone] = score.item()

Perimeter_top_ZN_3


  0%|          | 0/800 [00:00<?, ?it/s]

/Users/aakashsasikumar/Documents/MSc/Research/Dissimilar Policy/building-MARL/studies/../ope/sniw.py:60: RuntimeWarning: divide by zero encountered in log
  return np.log(value / (1- value))


Perimeter_bot_ZN_1


  0%|          | 0/800 [00:00<?, ?it/s]

...

  0%|          | 0/800 [00:00<?, ?it/s]

Perimeter_mid_ZN_1


  0%|          | 0/800 [00:00<?, ?it/s]

Perimeter_bot_ZN_4


  0%|          | 0/800 [00:00<?, ?it/s]

In [6]:
eval_data_loc = "../data/1month_eval.csv"
df = pd.read_csv(eval_data_loc, header=None, names=["datetime","policy","zone","energy"])
spearman_corr_sniw = {}
for zone in zones:
    eval_df = df[df["zone"] == zone]
    for i_policy in invalid_policies:
        eval_df = eval_df[eval_df["policy"]!=i_policy[3:]]
    eval_df = eval_df.sort_values(by=["energy"])
    score_list = []
    for i, row in eval_df.iterrows():
        score_list.append(policy_scores_sniw[f"../{row['policy']}"][zone])

    eval_df["ope_scores"] = score_list
    correlation = spearmanr(eval_df["energy"].values, eval_df["ope_scores"].values, nan_policy="omit")
    spearman_corr_sniw[zone] = correlation

In [7]:
spearman_corr_sniw

{'Perimeter_top_ZN_3': SpearmanrResult(correlation=0.12714740268290245, pvalue=0.016687339130026853),
 'Perimeter_bot_ZN_1': SpearmanrResult(correlation=0.06762521029252737, pvalue=0.20497397169052958),
 'Perimeter_top_ZN_1': SpearmanrResult(correlation=0.10507153874813083, pvalue=0.04854385192075492),
 'Core_bottom': SpearmanrResult(correlation=0.26865334943685487, pvalue=2.9889687860643267e-07),
 'Perimeter_top_ZN_4': SpearmanrResult(correlation=0.25494758309285576, pvalue=1.213159558832177e-06),
 'Core_top': SpearmanrResult(correlation=0.28735147749797374, pvalue=3.884233210605973e-08),
 'Perimeter_bot_ZN_2': SpearmanrResult(correlation=0.2391274544868194, pvalue=5.547545841098801e-06),
 'Perimeter_bot_ZN_3': SpearmanrResult(correlation=-0.15770110847420957, pvalue=0.0029679496457174263),
 'Perimeter_mid_ZN_4': SpearmanrResult(correlation=0.2777378623655178, pvalue=1.1302398439159801e-07),
 'Core_mid': SpearmanrResult(correlation=0.28380480091530236, pvalue=5.787354871167421e-08),
 

In [8]:
with open("snipw_raw_scores_30_days_15_07_2022.pkl", "wb+") as f:
    pickle.dump(policy_scores_sniw, f)

In [20]:
with open("snipw_spearman_corr_12_07_2022.pkl", "wb+") as f:
    pickle.dump(spearman_corr_sniw, f)

In [19]:
spearman_corr_sniw

{'Perimeter_top_ZN_3': SpearmanrResult(correlation=0.06832150697875147, pvalue=0.20098146026856523),
 'Perimeter_bot_ZN_1': SpearmanrResult(correlation=-0.000549627312431125, pvalue=0.9918134221787763),
 'Perimeter_top_ZN_1': SpearmanrResult(correlation=0.040329459228737256, pvalue=0.4513396850825411),
 'Core_bottom': SpearmanrResult(correlation=0.21938290898822832, pvalue=3.381951836216698e-05),
 'Perimeter_top_ZN_4': SpearmanrResult(correlation=0.20585597013228454, pvalue=0.00010248243553583014),
 'Core_top': SpearmanrResult(correlation=0.24155079420528947, pvalue=4.710911931915854e-06),
 'Perimeter_bot_ZN_2': SpearmanrResult(correlation=0.18566452252356164, pvalue=0.00047146866887882197),
 'Perimeter_bot_ZN_3': SpearmanrResult(correlation=-0.21674108922438642, pvalue=4.222820900497693e-05),
 'Perimeter_mid_ZN_4': SpearmanrResult(correlation=0.23155451685783632, pvalue=1.1737831499433819e-05),
 'Core_mid': SpearmanrResult(correlation=0.23708701541864868, pvalue=7.11697796335923e-06),